## Run CNMF source extraction on movies
Step 2 of the Caiman processing pipeline for dendritic two-photon calcium imaging movies. This part uses mmap files as input. These are created during motion correction with the Caiman toolbox (see `01_Preprocess_MC_3D.ipynb`). 

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [ ]:
# Generic imports
# from __future__ import absolute_import, division, print_function
# from builtins import *

import os, platform, glob, sys, re, copy
import fnmatch
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat

from IPython.display import clear_output

# Import Bokeh library
import bokeh.plotting as plotting
from bokeh.plotting import Figure, show
from bokeh.layouts import gridplot
from bokeh.models import Range1d, CrosshairTool, HoverTool, Legend
from bokeh.io import output_notebook, export_svgs
from bokeh.models.sources import ColumnDataSource

%matplotlib inline

In [ ]:
# This has to be in a separate cell, otherwise it wont work.
from bokeh import resources
output_notebook(resources=resources.INLINE)

In [ ]:
# on Linux we have to add the caiman folder to Pythonpath
if platform.system() == 'Linux':
    sys.path.append(os.path.expanduser('~/caiman'))
# environment variables for parallel processing
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['VECLIB_MAXIMUM_THREADS']='1'

In [ ]:
# CaImAn imports
import caiman as cm
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.components_evaluation import estimate_components_quality as estimate_q
from caiman.components_evaluation import estimate_components_quality_auto
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.source_extraction.cnmf import utilities as cnmf_utils
import caiman_utils as cm_utils
import utils as utils

### Select files
The following need to be specified:
- data_folder ... location of the data (relative to ~/Data)
- mc_output ... select if output of rigid ('rig') or piece-wise rigid ('els') motion correction should be used (currently only 'rig' is tested and works)
- max_files ... maximum number of files to process, e.g. for testing (if 0, all files will be processed)

In [ ]:
animal_folder = 'M3_October_2018'
date_folder = 'M3_2018-10-04'
session_folder = 'S1'
group_id = 'G0'

mc_output = 'rig' # mc output to use for source extract ('rig' or 'els')
remove_bad_frames = True # remove bad frames specified in Json file

n_processes = 8 # number of parallel processes (None to select automatically)

# create the complete path to the data folder
if platform.system() == 'Linux':
    data_folder = '/home/ubuntu/Data'
elif platform.system() == 'Darwin':
    data_folder = '/Users/Henry/Data/temp/Dendrites_Gwen'
data_folder = os.path.join(data_folder, animal_folder, date_folder, session_folder)

In [ ]:
# select the mmap file created during motion correction
all_files = os.listdir(data_folder)
mmap_files = sorted([x for x in all_files if x.startswith('%s_%s' % (date_folder, session_folder)) 
           and x.endswith('.mmap') and mc_output in x and group_id in x and not 'remFrames' in x])
n_planes = len(mmap_files)

print('Found %d mmap files. Check allocation to planes!' % (n_planes))
for i_plane in range(n_planes):
    print('Plane %d: %s' % (i_plane, mmap_files[i_plane]))

In [ ]:
mmap_files = [os.path.join(data_folder, x) for x in mmap_files]
# get metadata
for file in os.listdir(data_folder):
    if fnmatch.fnmatch(file, '%s_%s_Join_%s_*[!badFrames].json' % (date_folder, session_folder, group_id)):
        meta = json.load(open(os.path.join(data_folder,file)))
        break
trial_index = np.array(meta['trial_index'])
frame_rate = meta['frame_rate'] / n_planes

### Load data and remove bad frames

In [ ]:
bad_frames = np.array([], dtype='int64')
fname_list = []
images_list = []

# first, create list of bad frame indices (for all planes combined)
for fname in mmap_files:
    bad_frames = np.concatenate((bad_frames, cm_utils.getBadFrames(fname)))
bad_frames = np.unique(bad_frames)

# remove the bad frames from all files
for fname in mmap_files:
    Yr, dims = cm_utils.loadData(fname)
    images, Y, fname_rem, bad_frames_by_trial, trial_idx = cm_utils.removeBadFrames(fname, 
                                                                                      trial_index, 
                                                                                      Yr, dims, bad_frames, 
                                                                                      data_folder)
    fname_list.append(fname_rem)
    images_list.append(images)
trial_index = trial_idx

### Display frame average for each plane

In [ ]:
plt.figure(figsize=(30,60))
for ix_plane in range(n_planes):
    avg_img = np.mean(images_list[ix_plane],axis=0)
    plt.subplot(1,n_planes,ix_plane+1)
    plt.imshow(avg_img, cmap='gray'), plt.title('Frame average - Plane %d' % (ix_plane));

### Specify if plane contains dendritic signals
CaImAn uses different initialization methods depending on whether the signals are dendritic or somatic. Therefore, we need to specify the types of signal expected in each plane.

In [ ]:
is_dendritic = [True, False, True]

### Setup cluster
The default backend mode for parallel processing is through the multiprocessing package. This will allow us to use all the cores in the VM.

In [ ]:
# start the cluster (if a cluster already exists terminate it)
if 'dview' in locals() and dview is not None:
    dview.terminate()
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)

### Parameters for source extraction
Next, we define the important parameters for calcium source extraction. These parameters will have to be iteratively refined for the respective datasets.


In [ ]:
# dataset dependent parameters
decay_time = 0.4                            # length of a typical transient in seconds

# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system
gnb = 2                     # number of global background components
merge_thresh = 0.8          # merging threshold, max correlation allowed
rf = [25, 50]                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
stride_cnmf = 5             # amount of overlap between the patches in pixels
K = 20                       # number of components per patch
gSig = [4, 4]               # expected half size of neurons in pixels

method_init = 'sparse_nmf'  # initialization method (if analyzing dendritic data use 'sparse_nmf', else 'greedy_roi')
#alpha_snmf = 10e2           # sparsity penalty for dendritic data analysis through sparse NMF
alpha_snmf = 100
normalize_init = True      # default is True

ssub = 1                    # spatial subsampling during initialization
tsub = 1                    # temporal subsampling during intialization

In [ ]:
# create Parameters object
# unspecified parameters get default values
opts_dict = {'fnames': fname_rem,
            'fr': frame_rate,
            'decay_time': decay_time,
            'p': 1,
            'nb': gnb,
            'rf': rf,
            'K': K, 
            'stride': stride_cnmf,
            'method_init': method_init,
            'alpha_snmf': alpha_snmf,
             'normalize_init': normalize_init,
            'rolling_sum': True,
            'only_init': True,
            'ssub': ssub,
            'tsub': tsub}

opts = params.CNMFParams(params_dict=opts_dict)

To get a dict with all parameters, use `opts.to_dict()`

#### Run CNMF on patches

In [ ]:
# First extract spatial and temporal components on patches and combine them
# for this step deconvolution is turned off (p=0)
# Then re-run seeded CNMF on accepted patches to refine and perform deconvolution
opts.set('temporal', {'p': 0})
cnm_list = []

t_start = time.time()
for ix_plane in range(n_planes):
    opts_plane = copy.deepcopy(opts)
    if is_dendritic[ix_plane]:
        opts_plane.set('init', {'method_init': 'sparse_nmf'})
    else:
        opts_plane.set('init', {'method_init': 'greedy_roi'})
    cnm = cnmf.CNMF(n_processes, params=opts_plane, dview=dview)
    cnm.fit(images_list[ix_plane])
     
    cnm.params.set('temporal', {'p': p})
    cnm2 = cnm.refit(images_list[ix_plane], dview=dview)
    
    cnm_list.append(cnm2)
    
    clear_output()
    
t_elapsed = time.time() - t_start
print('\nFinished Source Extract in %1.2f s' % (t_elapsed))

### Evaluate components

In [ ]:
# Parameters for evaluation
quality_params = {
    'min_SNR': 2.5,               # signal to noise ratio for accepting a component
    'rval_thr': 0.95,              # space correlation threshold for accepting a component
    'cnn_thr': 0.99,              # threshold for CNN based classifier
    'cnn_lowest': 0.5            # neurons with cnn probability lower than this value are rejected
}

for ix_plane, cnm in enumerate(cnm_list):
    opts = copy.deepcopy(cnm.params)
    opts.set('quality', quality_params)
    cnm.estimates.evaluate_components(images_list[ix_plane], opts, dview=dview)
    cnm_list[ix_plane] = cnm
    print('\nPlane %d' % (ix_plane))
    print('Found %d good / %d bad components\n' % (len(cnm.estimates.idx_components), 
                                                 len(cnm.estimates.idx_components_bad)))

### Save CNMF results
After the time consuming steps of the source extraction are completed, it makes sense to store the results. Variables are stored in the Numpy-specific `.npz` format.

In [ ]:
npz_name = os.path.join(data_folder, '%s_%s_Join_%s_results_CNMF.npz' % (date_folder, session_folder, group_id))

nb_params = {
    'data_folder': data_folder,
    'date_folder': date_folder,
    'session_folder': session_folder,
    'group_id': group_id,
    'meta': meta,
    'trial_index': trial_index,
    'bad_frames': bad_frames,
    'bad_frames_by_trial': bad_frames_by_trial
}

for ix_plane, cnm in enumerate(cnm_list):
    cnm.dview = None
    cnm_list[ix_plane] = cnm
np.savez(npz_name, cnm_list=cnm_list, images_list=images_list, nb_params=nb_params)
print('Saved CNMF results in %s' % (npz_name))

### Load CNMF results
Load results from a previous CNMF run. If the analysis is continued right away, the load step can be skipped.

If you reload the notebook, you have to import the required modules (i.e. execute cells 1 - 4).

In [ ]:
load_data = True

if load_data:
    npz_name = '/Users/Henry/Data/temp/Dendrites_Gwen/M3_October_2018/M3_2018-10-04/S1/M3_2018-10-04_S1_Join_G0_results_CNMF.npz'
    npz_content = np.load(npz_name)
    nb_params = npz_content['nb_params'][()]
    cnm_list = npz_content['cnm_list'][()]
    images_list = npz_content['images_list'][()]

    print('Loaded file %s (%d planes)' % (os.path.basename(npz_name), len(cnm_list)))
else:
    nb_params = {
        'data_folder': data_folder,
        'date_folder': date_folder,
        'session_folder': session_folder,
        'group_id': group_id,
        'meta': meta,
        'trial_index': trial_index,
        'bad_frames': bad_frames,
        'bad_frames_by_trial': bad_frames_by_trial
}

Pre-compute local correlations as they will be used a lot.

In [ ]:
Cn = []
for img in images_list:
    cc = cm.local_correlations(img.transpose(1,2,0))
    cc[np.isnan(cc)] = 0
    Cn.append(cc)

Plot contours of selected and rejected components

In [ ]:
for ix_plane, cnm in enumerate(cnm_list):
    print('Plane %d' % (ix_plane))
    cnm.estimates.plot_contours_nb(img=Cn[ix_plane], idx=cnm.estimates.idx_components)

View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using: 'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
for ix_plane, cnm in enumerate(cnm_list):
    print('Plane %d' % (ix_plane))
    cnm.estimates.nb_view_components(img=Cn[ix_plane], idx=cnm.estimates.idx_components)

In [ ]:
# rejected components
for ix_plane, cnm in enumerate(cnm_list):
    print('Plane %d' % (ix_plane))
    if len(cnm.estimates.idx_components_bad) > 0:
        cnm.estimates.nb_view_components(img=Cn[ix_plane], idx=cnm.estimates.idx_components_bad)
    else:
        print("No components were rejected.")

### Accessing parameters in the CNM object
This cell shows how to access relevant parameters stored in the `cnm` object.

```python
A, C, b, f, YrA, S, sn = cnm.estimates.A, cnm.estimates.C, cnm.estimates.b, cnm.estimates.f, cnm.estimates.YrA, cnm.estimates.S, cnm.estimates.sn
```

<hr>

**Explanation of parameters:**
- A   ... n_pixel x n_components sparse matrix (component locations)
- C   ... n_component x t np.array (fitted signal)
- b   ... ? np.array
- f   ... ? np.array (b / f related to global background components)
- YrA ... n_component x t np.array (residual)
- S   ... deconvolved signal (spike rate(ish))
- sn  ... n_pixel np.array (SNR?)

<hr>

**Convert sparse component matrix to dense matrix:**
```python
A_dense = A.todense()
```

<hr>

**Indices of good and bad components:**
``` python
idx_comps = cnm.estimates.idx_components
idx_comps_bad = cnm.estimates.idx_components_bad
```

<hr>

### Select components
First, create a plot with good components on background image and as component map. This plot is saved as a PNG file in the data folder.

In [ ]:
for ix_plane, cnm in enumerate(cnm_list):
    avg_img = np.mean(images_list[ix_plane],axis=0)
    idx_comps = cnm.estimates.idx_components
    A = cnm.estimates.A
    A_dense = A.todense()

    counter = 1
    plt.figure(figsize=(20,40));
    for i_comp in range(len(idx_comps)):
        plt.subplot(len(idx_comps),2,counter)
        if counter == 1:
            plt.title('CNMF Components - Plane %d' % (ix_plane), fontsize=24);
        
        counter += 1
        dummy = cm.utils.visualization.plot_contours(A[:,idx_comps[i_comp]], avg_img, cmap='gray', 
                                                     colors='r', display_numbers=False)
        component_img = np.array(np.reshape(A_dense[:,idx_comps[i_comp]], avg_img.shape, order='F'))
        plt.subplot(len(idx_comps),2,counter)
        counter += 1
        plt.imshow(component_img), plt.title('Component %1.0f' % (i_comp))
 
    plt.tight_layout()

    fig_name = os.path.join(nb_params['data_folder'], '%s_%s_Join_%s_P%d_Components.png' % 
                            (nb_params['date_folder'], nb_params['session_folder'], nb_params['group_id'], ix_plane))
    plt.savefig(fig_name)
    plt.close()
    
    print('Saved file %s' % (fig_name))

Next, choose components that should be removed from the list of good components.

In [ ]:
for ix_plane, cnm in enumerate(cnm_list):

    print('\n\nComponent selection - Plane %d' % (ix_plane))
    cnm = cnm_list[ix_plane]

    idx_comps = cnm.estimates.idx_components
    idx_comps_bad = cnm.estimates.idx_components_bad

    response = input('\nSelect indices of good components to exclude (comma-separated list):\n')

    if len(response) > 0:
        comps_to_exclude = [int(x) for x in response.split(',')]

        # add to bad components
        idx_comps_bad = np.sort(np.append(idx_comps_bad, idx_comps[comps_to_exclude]))
        # remove from good components
        idx_comps = np.delete(idx_comps, comps_to_exclude)

        # after re-classification
        print('Good components: ')
        print(idx_comps)
        print('Bad components: ')
        print(idx_comps_bad)

        # save back components
        cnm_list[ix_plane].estimates.idx_components = idx_comps
        cnm_list[ix_plane].estimates.idx_components_bad = idx_comps_bad

Create component matrices with good components

In [ ]:
component_matrix_list = []
for ix_plane, cnm in enumerate(cnm_list):
    idx_comps = cnm.estimates.idx_components
    A = cnm.estimates.A
    A_dense = A.todense()
    
    for i_comp in range(len(idx_comps)):
        component_img = np.array(np.reshape(A_dense[:,idx_comps[i_comp]], cnm.dims, order='F'))
        if i_comp == 0:
            component_matrix = component_img
        else:
            component_matrix = np.dstack((component_matrix, component_img))
            
    component_matrix_list.append(component_matrix)

#### Extract DF/F values and discard bad components
The CaImAn function `detrend_df_f` uses a sliding window percentile filter to determine the baseline and compute DFF.
Note: for noisy traces and / or high levels of activity, `detrend_df_f` seems to produce sometimes unexpected results (i.e. trace whose shape differs a lot from the extracted component traces). It might be better to use the extracted component traces (see below) for downstream analysis.

In [ ]:
for ix_plane, cnm in enumerate(cnm_list):
    cnm.estimates.select_components(use_object=True)
    cnm.estimates.detrend_df_f(quantileMin=8, frames_window=250) # results are in cnm.estimates.F_dff
    cnm_list[ix_plane] = cnm

Interactive plot of selected components

In [ ]:
for ix_plane, cnm in enumerate(cnm_list):
    print('Plane %d' % (ix_plane))
    cnm.estimates.nb_view_components(img=Cn[ix_plane], denoised_color='red')

In [ ]:
meta = nb_params['meta']
source_files = meta['source_file']
source_frames = np.array(meta['source_frames'])
trial_index = nb_params['trial_index']

# get corresponding trial name for each frame
trial_names = [x.replace('_crop.tif','') for x in source_files]
trial_names_frames = [trial_names[x] for x in trial_index]

### Create stacked plot of components
Plot stacked traces for some or all components. The source data and plane can be selected. The plot also shows the trial for each frame.
Types of source data that can be plotted:
- F_dff ... detrended DF/F
- YrA ... residual
- C ... denoised signal
- Y_r ... ROI signal (C + YrA)
- S ... Deconvolved signal

In [ ]:
comp_idx = [0,1,2,3,4,5,6] # select index of components to plot, e.g. [0,1,2] / use None to plot all components
source = 'F_dff' # select the data that should be plotted ('F_dff', 'Y_r', 'C', 'S', 'YrA')
ix_plane = 0 # select plane that should be plotted

cnm = cnm_list[ix_plane]

if source == 'F_dff':
    source_data = cnm.estimates.F_dff
elif source == 'Y_r':
    source_data = cnm.estimates.YrA + cnm.estimates.C
elif source == 'YrA':
    source_data = cnm.estimates.YrA
elif source == 'C':
    source_data = cnm.estimates.C
elif source == 'S':
    source_data = cnm.estimates.S
else:
    raise Exception('Specified source_data is not implemented')

t = np.arange(0, source_data.shape[-1]) / meta['frame_rate']
    
if comp_idx is not None:
    source_data = source_data[comp_idx,:]

p = Figure(plot_width=900, plot_height=600, title=('%s %s CNMF Results' % (nb_params['date_folder'], nb_params['session_folder'])))    
legend_text = ['Component %1d' % (x) for x in range(source_data.shape[0])]

# this is the call to the plotting function (change args. as required)
utils.plotTimeseries(p, t, source_data, legend=legend_text, stack=True, xlabel='Time [s]', ylabel=source,
                     output_backend='canvas', trial_index=trial_index, trial_names_frames=trial_names_frames)

### Split up by trials and save as .mat

In [ ]:
# First, check if number of frames match
if not (np.sum(source_frames)-len(nb_params['bad_frames'])) == cnm_list[0].estimates.F_dff.shape[-1]:
    raise Exception('Sum of source frames minus number of bad frames must be equal to number of timepoints.')

In [ ]:
bad_frames_by_trial = nb_params['bad_frames_by_trial']

for ix_plane, cnm in enumerate(cnm_list):

    results_dff = dict()
    results_Yr = dict()
    results_C = dict()
    results_S = dict()
    removed_frames = dict()
    
    F_dff = cnm.estimates.F_dff
    Y_r = cnm.estimates.YrA + cnm.estimates.C
    C = cnm.estimates.C
    S = cnm.estimates.S
    
    for ix, trial_file in enumerate(source_files):
        # get indices for current trial's frames
        trial_indices = np.where(trial_index==ix)[0]

        if ix in bad_frames_by_trial:
            removed_frames_trial = bad_frames_by_trial[ix]
        else:
            removed_frames_trial = []

        # create a valid Matlab variable / field name
        field_name = str('x' + source_files[ix][:source_files[ix].find('/')]).replace('_Live','').replace('-','_')
        results_dff[field_name] = F_dff[:,trial_indices]
        results_Yr[field_name] = Y_r[:,trial_indices]
        results_C[field_name] = C[:,trial_indices]
        results_S[field_name] = S[:,trial_indices]
        removed_frames[field_name] = removed_frames_trial
        
    # dictionary for saving as mat file (field names will be variable names in Matlab)
    mdict = {
        'trials': [str(x) for x in source_files], 
        'dff_trial': results_dff,
        'Yr_trial': results_Yr,
        'C_trial': results_C,
        'Deconv_trial': results_S,
        'removed_frames': removed_frames,
        'mean_image': np.mean(images_list[ix_plane],axis=0),
        'spatial_components': component_matrix_list[ix_plane],
        'local_correlations': Cn[ix_plane]
      }
    
    # save the .mat file
    matfile_name = os.path.join(nb_params['data_folder'], '%s_%s_Join_%s_P%d_results_CNMF.mat' % 
                                (nb_params['date_folder'], nb_params['session_folder'], nb_params['group_id'], ix_plane))
    savemat(os.path.join(nb_params['data_folder'], matfile_name), mdict=mdict, long_field_names=True)